### Solar 10.7B Instruct

This notebook demonstrates the use of LlamaIndex for Retrieval Augmented Generation in Linux and with Nvidia's CUDA.

See the [README.md](README.md) file for help on how to run this.

#### 1. Prepare Llama Index for use

In [1]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import VectorStoreIndex
from llama_index import SimpleDirectoryReader
from llama_index import ServiceContext

INFO:numexpr.utils:Note: NumExpr detected 24 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
Note: NumExpr detected 24 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
NumExpr defaulting to 8 threads.


#### 2. Load the Word document(s)

Note: A fictitious story about Thundertooth a dinosaur who has travelled to the future. Thanks ChatGPT!

In [2]:
documents = SimpleDirectoryReader("./Data/").load_data()

#### 3. Instantiate the model

In [3]:
import torch

from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt
llm = LlamaCPP(
    model_url=None, # We'll load locally.
    model_path='./Models/solar-10.7b-instruct-v1.0.Q5_K_M.gguf',
    temperature=0.1,
    max_new_tokens=1024, # Increasing to support longer responses
    context_window=4096, # Context Window
    generate_kwargs={},
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 49}, # 49 was all that was needed and that fit on the RTX 3090's 24GB
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True
)

ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: NVIDIA GeForce RTX 3090, compute capability 8.6
llama_model_loader: loaded meta data with 24 key-value pairs and 435 tensors from ./Models/solar-10.7b-instruct-v1.0.Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q5_K     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q6_K     [ 14336,  4096,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q5_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q5_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_m

#### 4. Checkpoint

Are you running on GPU? The above output should include near the top something like:
> ggml_init_cublas: found 1 CUDA devices:

And in the full text near the bottom should be:
> llm_load_tensors: using CUDA for GPU acceleration

#### 5. Embeddings

Convert your source document text into embeddings.

The embedding model is from huggingface, this one performs well.

> https://huggingface.co/thenlper/gte-large


In [4]:
from langchain.embeddings import HuggingFaceEmbeddings

embed_model = HuggingFaceEmbeddings(model_name="thenlper/gte-large", cache_folder=None)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: thenlper/gte-large
Load pretrained SentenceTransformer: thenlper/gte-large
INFO:sentence_transformers.SentenceTransformer:Did not find folder thenlper/gte-large
Did not find folder thenlper/gte-large
INFO:sentence_transformers.SentenceTransformer:Search model on server: http://sbert.net/models/thenlper/gte-large.zip
Search model on server: http://sbert.net/models/thenlper/gte-large.zip
INFO:sentence_transformers.SentenceTransformer:Load SentenceTransformer from folder: /home/mark/.cache/torch/sentence_transformers/sbert.net_models_thenlper_gte-large
Load SentenceTransformer from folder: /home/mark/.cache/torch/sentence_transformers/sbert.net_models_thenlper_gte-large
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cuda
Use pytorch device: cuda


#### 6. Prompt Template

Prompt template for SOLAR 10.7B Instruct:

&#35;&#35;&#35; User:<br>
{prompt}<br>

&#35;&#35;&#35; Assistant:

In [5]:
# Produces a prompt specific to the model
def modelspecific_prompt(promptmessage, questionText):
    return f"### User:\n{promptmessage}{questionText}\n\n###Assistant:"

#### 7. Service Context

For chunking the document into tokens using the embedding model and our LLM

In [6]:
service_context = ServiceContext.from_defaults(
    chunk_size=256, # Number of tokens in each chunk
    llm=llm,
    embed_model=embed_model
)

#### 8. Index documents

In [7]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

#### 9. Query Engine

Create a query engine, specifying how many citations we want to get back from the searched text (in this case 3).

The DB_DOC_ID_KEY is used to get back the filename of the original document

In [8]:
from llama_index.query_engine import CitationQueryEngine
query_engine = CitationQueryEngine.from_args(
    index,
    similarity_top_k=3,
    # here we can control how granular citation sources are, the default is 512
    citation_chunk_size=256,
)

# For citations we get the document info
DB_DOC_ID_KEY = "db_document_id"

#### 10. Prompt and Response function

Pass in a question, get a response back.

IMPORTANT: The prompt is set here, adjust it to match what you want the LLM to act like and do.

In [9]:
def RunQuestion(questionText):
    systemmessage = "You are a story teller who likes to elaborate. Answer questions in a positive, helpful and interesting way. If the answer is not in the following context return ONLY 'Sorry, I don't know the answer to that'. Question: "

    queryQuestion = modelspecific_prompt(systemmessage, questionText)

    response = query_engine.query(queryQuestion)

    return response

#### 11. Questions to test with

In [10]:
TestQuestions = [
    "Summarise the story for me",
    "Who was the main protagonist?",
    "Did they have any children? If so, what were their names?",
    "Did anything eventful happen?",
]

#### 12. Run Questions through model (this can take a while) and see citations

Runs each test question, saves it to a dictionary for output in the last step.

Note: Citations are the source documents used and the text the response is based on. This is important for RAG so you can reference these documents for the user, and to ensure it's utilising the right documents.

In [11]:
qa_pairs = []

for index, question in enumerate(TestQuestions, start=1):
    question = question.strip() # Clean up

    print(f"\n{index}/{len(TestQuestions)}: {question}")

    response = RunQuestion(question) # Query and get  response

    qa_pairs.append((question.strip(), str(response).strip())) # Add to our output array

    # Displays the citations
    for index, node in enumerate(response.source_nodes, start=1):
        print(f"{index}/{len(response.source_nodes)}: |{node.node.metadata['file_name']}| {node.node.get_text()}")

    # Uncomment the following line if you want to test just the first question
    # break 


1/4: Summarise the story for me


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

1/4: |Thundertooth Part 1.docx| Source 1:
"Hello there, majestic creature. What brings you to our time?" Mayor Grace inquired, her voice calm and reassuring.



Thundertooth, though initially startled, found comfort in the mayor's soothing tone. In broken sentences, he explained his journey through time, the strange portal, and his hunger dilemma. Mayor Grace listened intently, her eyes widening with amazement at the tale of the prehistoric dinosaur navigating the future.



Realizing the dinosaur's predicament, Mayor Grace extended an invitation. "You are welcome in our city, Thundertooth. We can find a way to provide for you without causing harm to anyone. Let us work together to find a solution."



Grateful for the mayor's hospitality, Thundertooth followed her through the city. Together, they explored the futuristic marketplaces and innovative food labs, eventually discovering a sustainable solution that satisfied the dinosaur's hunger without compromising the well-being of the ci


llama_print_timings:        load time =    2293.84 ms
llama_print_timings:      sample time =     118.41 ms /   416 runs   (    0.28 ms per token,  3513.10 tokens per second)
llama_print_timings: prompt eval time =    8169.06 ms /  1338 tokens (    6.11 ms per token,   163.79 tokens per second)
llama_print_timings:        eval time =   40271.79 ms /   415 runs   (   97.04 ms per token,    10.30 tokens per second)
llama_print_timings:       total time =   49422.15 ms


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...


llama_print_timings:        load time =    2293.84 ms
llama_print_timings:      sample time =      43.06 ms /   156 runs   (    0.28 ms per token,  3623.02 tokens per second)
llama_print_timings: prompt eval time =    7355.09 ms /  1086 tokens (    6.77 ms per token,   147.65 tokens per second)
llama_print_timings:        eval time =   14203.57 ms /   155 runs   (   91.64 ms per token,    10.91 tokens per second)
llama_print_timings:       total time =   21912.05 ms


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Llama.generate: prefix-match hit


1/4: |Thundertooth Part 1.docx| Source 1:
"Hello there, majestic creature. What brings you to our time?" Mayor Grace inquired, her voice calm and reassuring.



Thundertooth, though initially startled, found comfort in the mayor's soothing tone. In broken sentences, he explained his journey through time, the strange portal, and his hunger dilemma. Mayor Grace listened intently, her eyes widening with amazement at the tale of the prehistoric dinosaur navigating the future.



Realizing the dinosaur's predicament, Mayor Grace extended an invitation. "You are welcome in our city, Thundertooth. We can find a way to provide for you without causing harm to anyone. Let us work together to find a solution."



Grateful for the mayor's hospitality, Thundertooth followed her through the city. Together, they explored the futuristic marketplaces and innovative food labs, eventually discovering a sustainable solution that satisfied the dinosaur's hunger without compromising the well-being of the ci


llama_print_timings:        load time =    2293.84 ms
llama_print_timings:      sample time =      91.54 ms /   325 runs   (    0.28 ms per token,  3550.36 tokens per second)
llama_print_timings: prompt eval time =    8066.00 ms /  1049 tokens (    7.69 ms per token,   130.05 tokens per second)
llama_print_timings:        eval time =   30532.97 ms /   324 runs   (   94.24 ms per token,    10.61 tokens per second)
llama_print_timings:       total time =   39373.70 ms


#### 13. Output responses

In [12]:
for index, (question, answer) in enumerate(qa_pairs, start=1):
    print(f"{index}/{len(qa_pairs)} {question}\n\n{answer}\n\n--------\n")

1/4 Summarise the story for me

In this captivating tale, we encounter Thundertooth, a prehistoric dinosaur who is mysteriously transported through time and finds himself in a futuristic city. Upon his arrival, he encounters Mayor Grace, who warmly welcomes him despite the astonishment of such an unexpected visitor. Thundertooth shares his incredible journey and expresses his concern about finding food as a carnivorous dinosaur in this modern setting.

Mayor Grace, showing empathy and understanding, assures Thundertooth that he is safe within her city's boundaries and they will work together to find a solution for his hunger without causing harm to the citizens. They explore advanced food labs and futuristic marketplaces, eventually discovering a sustainable way to satisfy Thundertooth's needs.

As news of Thundertooth's arrival spreads, the city embraces him as a symbol of unity between past and future. He becomes an integral part of the community, living in the park where holographic